In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel


In [2]:
list=[[20191008],[20191008],[20191008]]
df= spark.createDataFrame(list,['tst'])

df=df.withColumn("tst", F.col("tst").cast("string"))
df.show()

+--------+
 tst|
+--------+
20191008|
20191008|
20191008|
+--------+

In [3]:
df.withColumn("tst", F.to_date("tst", 'yyyyMMdd')).show()

+----------+
 tst|
+----------+
2019-10-08|
2019-10-08|
2019-10-08|
+----------+

In [4]:
df1=df.withColumn("Date", F.to_date("Date")).orderBy('Date')

In [5]:
from datetime import datetime

target_date= '2020-01-01'

window_row = Window.partitionBy('I1','Id2','Date')

df2=df1.withColumn('max_Date', when((col('Date')<=target_date),max('Date').over(window_row)))
df3=df2.withColumn('cum_sum', when((col('Date')==col('max_Date')),sum('Sale').over(window_row)))
df3.show()

+---+---+----------+----+----------+-------+
 I1|Id2| Date|Sale| max_Date|cum_sum|
+---+---+----------+----+----------+-------+
 BB| C0|2019-03-01| 1|2019-03-01| 1|
 CC| B0|2019-03-01| 8|2019-03-01| 8|
 CC| B0|2019-04-01| 1|2019-04-01| 1|
 BB| C0|2019-05-01| 26|2019-05-01| 26|
 AA| B0|2019-07-01| 1|2019-07-01| 1|
 AA| B0|2020-01-01| 23|2020-01-01| 25|
 AA| B0|2020-01-01| 2|2020-01-01| 25|
 AA| B0|2020-02-01| 0| null| null|
 BB| C0|2020-03-01| 1| null| null|
 AA| B0|2020-12-01| 116| null| null|
+---+---+----------+----+----------+-------+